# Importing libraries

In [63]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import pickle

#nltk.download('all')

# Reading Corpus

In [64]:
path = "corpus/"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))

# Removing tags, headers, Lower Casing and tokenization 

In [66]:
docs_words = {}
i = 0 #for docs_words

for f in files:
    doc = open(f, errors='ignore')
    html = doc.read()
    
    index = html.find("<html")
    if index == -1 :
        html.find("<Html")
    elif index == -1 : 
        html.find("<HTML")
    html = html[index:]

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    #print("Doc : " + text + "\n\n")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(text)
    words_pos = {}
    j = 0; #for words_pos
    for t in tokens:
        t = t.lower() #lowering Case
        
        if t not in words_pos.keys():
            words_pos[t] = []
        if t in words_pos.keys():
            words_pos[t].append(j)
        j = j + 1
    
    docs_words[i] = words_pos
    i = i + 1

#print(docs_words)

# Removing Stop words

In [67]:
f = open("stoplist.txt", "r") 
stoplist = f.read().splitlines() #Stoplist words

for i in range(len(docs_words)):
    for key in list(docs_words[i]):
        if key in stoplist or len(key) == 1:
            docs_words[i].pop(key,None)
#print(docs_words)            

# Stemming

In [68]:
ps = PorterStemmer()
for i in range(len(docs_words)):
    for key in list(docs_words[i]):
        stem = ps.stem(key)
        stem = stem.replace("'", "")
        if stem != key:
            if stem not in docs_words[i].keys():
                docs_words[i][stem] = docs_words[i].pop(key)
            elif stem in docs_words[i].keys():
                docs_words[i][stem] = docs_words[i][stem] + docs_words[i].pop(key)
            
#print(docs_words)

# Inverted Index using Dictionary

In [69]:
words_docs = {}

for i in range(len(docs_words)):
    for key in docs_words[i].keys():
        docs_pos = {}
        docs_pos[i] = docs_words[i].get(key)
        if key not in words_docs.keys():
            words_docs[key] = docs_pos
        elif key in words_docs.keys():
             words_docs[key][i] = docs_pos[i]

#print(words_docs)

# Inverted Index WITHOUT Dictionary

In [70]:
list_of_tuples = []
i = 0

for key in words_docs.keys():
    for docids in words_docs[key].keys():
        tup = (i, docids)
        list_of_tuples.append(tup)
    i = i + 1    

# Finding total occurences of a term

In [71]:
def get_total_occurences(key):
    count = 0
    
    for docid in words_docs[key].keys():
        count = count + len(words_docs[key][docid]) 
    return count    

# Writing term_index.txt

In [72]:
term_index = open("term_index.txt", "w", errors="ignore")
i = 0
term_info = {}

for key in words_docs.keys():
    ocr = get_total_occurences(key)
    docs = len(words_docs[key].keys())
    tup = (i, ocr, docs)
    term_info[key] = tup
    
    term_index.write(str(i) + "\t" + str(ocr) + "\t" + str(docs) + "\t" + str(words_docs[key]) + "\n")
    i = i + 1

# Making pickles of Inverted Index and Term info

In [74]:
with open('Inverted-Index.pickle', 'wb') as handle:
    pickle.dump(words_docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Term-Info.pickle', 'wb') as handle:
    pickle.dump(term_info, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [75]:
with open('Inverted-Index.pickle', 'rb') as handle:
    a = pickle.load(handle)
    
with open('Term-Info.pickle', 'rb') as handle:
    b = pickle.load(handle)   